# Метод максимального правдоподобия
## Теоретические основы

Метод максимального правдоподобия (maximum likelihood estimation - MLE) является одним из наиболее важных как в теоретическом, так и в практическом смыслах методом в машинном обучении. 

Решение задачи машинного обучения в общих чертах может быть представлено следующей схемой:
1. выбирается соответсвующая задаче модель (линейная регрессия, логистическая регрессия и т.п.) $f(X,\theta)$;
2. выбирается функция потерь для оптимизации $L(\theta,X,y)$;
3. выбирается метод оптимизации;
4. выбираются данные для обучения $X,y$;

После этого обучается модель: рассчитываются минимизирующие функцию потерь параметры $\theta$ выбранной модели:
$$\theta_{opt} = \arg \min L(\theta,X,y)$$

Для простоты и наглядности мы взяли задачу *обучения с учителем*, но подчеркнем, что нет никакой принципиальной трудности рассмотреть и методы без учителя.

Можно сказать, что выбор во всех четырех пунктах является до некоторой степени творческой задачей. При этом пункт 2) "функция потерь" нужно выделить отдельно: именно функция потерь свидетельствует о хорошем или плохом качестве решения всей задачи машинного обучения.

До текущего момента функцию потерь мы выбирали лишь из "общих соображений": малое значение суммы квадратов $\sum_i (y_i - f(X_i,\theta))^2$ свидетельствует о близости модельных и реальных данных в задаче регрессии, малое значение кросс-энтропии $\sum_i (y_i\ln \sigma(X_i,\theta) + (1-y_i)\ln (1-\sigma(X_i,\theta))$ свидетельствует о высокой точности меток в задаче классификации. Но есть ли более сильное обоснование выбора функции потерь?

Здесь мы поговорим об обоснованиях выбора той или иной функции потерь с точки зрения краеугольных дисциплин для машинного обучения - теории вероятностей и математической статистики.

Пусть дан датасет $X$ - признаки и $y$ - разметка или измерение (в дальнейшем будем говорить измерение). Для начала зададим себе вопрос: может ли в i-том примере $(X_i,y_i)$ содержаться некоторая случайность? Ответ на этот вопрос очевиден: да, конечно может и абсолютно точно (исключая тривиальные случаи) содержится. Случайности главным образом проявляются:
1. в самих данных $X_i$. Можно сказать, что данные принадлежат некоторому вероятностному распределению $P(X_i)$. Действительно, наличие тех или иных объектов на фото, определенных слов в тексте является по сути случайным, поэтому мы и говорим о распределении $P(X_i)$. Отметим, что распределение $P(X_i)$ может быть как дискретным, так и непрерывным;
2. в ошибках измерений $y_i$. Все ошибаются, в том числе и те, кто измеряет (человек или прибор). Основные ошибки измерений происходят в контексте с данными $X_i$. То есть, например, человек принимает собаку за кошку на фотографии не заранее подбрасывая монетку, а по причине того, что кошка на фотографии действительно может сильно напоминать собаку. Это дает нам право ввести условную вероятность $p(y_i|X_i)$. В случае регрессии можно говорить об условной плотности $p(y_i|X_i)$.

Сразу отметим, что для практических целей условную вероятность (или плотность) $p(y_i|X_i)$ удобно считать параметрической, то есть $p(y_i|X_i) = p(y_i|X_i,\theta)$, то есть при изменении параметров $\theta$ изменяется и $p(y_i|X_i,\theta)$.

Помимо условной вероятности (плотности) i-того примера $p(y_i|X_i,\theta)$ следует ввести совместную вероятность (плостность) всех примеров $p(y|X,\theta) = p(y_1,y_2,...,y_i,...,y_n|X,\theta)$. Если считать каждый i-тый пример независимым от остальных, то совместная вероятность (плотность) немедленно определяется как $p(y|X,\theta) = \prod_{i=1}^{n} p(y_i|X_i,\theta)$

После введения совместной условной вероятности (или плотности) $p(y|X,\theta)$ мы можем непосредственно перейти к определению правдоподобия.

Пусть дана совместная вероятность (плотность) $p(y|X,\theta)$. Если в $p(y|X,\theta)$ подставить $X,y$ из заданного  датасета, то получится функция $L$, зависящая только от параметров $\theta: L(\theta)$. Функция $L(\theta)$ называется **функцией правдоподобия**. Таким образом, функция правдоподобия при подстановке конкретных параметров $\theta$ это вероятность (значение плотности вероятности) наблюдать заданные значения $y$ при заданных признаках $X$. Следовательно, при различных параметрах функция правдоподобия может принимать бОльшие или меньшие значения. Мы вплотную подошли к *методу максимального правдоподобия*.

*Метод максимального правдоподобия* заключается в поиске таких параметров $\theta$, которые, собственно, и максимизируют функцию правдоподобия $L(\theta)$:
$$\theta_{MLE}=\arg\max L(\theta)$$
Это кажется очень здравой идеей. Действительно, метод максимального правдоподобия фактически позволяет найти параметры $\theta$, при которых наблюдать заданный вектор измерений $y$ при заданном $X$ становится наиболее вероятно, что полностью соотносится с концептом машинного обучения в смысле "объяснения" признаками $X$ измерений $y$.

Для многих практически значимых случаев гораздо лучше оптимизровать не саму функцию правдоподобия $L(\theta)$, а логарифм от нее $\ln L(\theta)$, что не меняет точки максимума $\theta_{MLE}$, но может существенно облегчить вычисления:
$$\theta_{MLE}=\arg\max \ln L(\theta)$$

Остается вопрос, как быть с новыми данными? Пусть имеется новый объект с признакми $X_{new}$ и полученная в результате применения MLE оценка плотности $p(y_{new}|X_{new},\theta_{MLE})$. Тогда со многих точек зрения наиболее удачным выбором для оценки $\tilde{y}_{new}$ будет просто математичсекое ожидание: $\tilde{y}_{new}=E(y_{new}|X_{new},\theta_{MLE})$.

## Вывод МНК через MLE

Давайте займемся практическим выводом некоторых важнейших функций потерь. Начнем с МНК в линейной регрессии.

Заметим, что в задачах регрессии мы имеем дело с плотностями распределений вероятностей.

Чтобы продолжить разговор в терминах, введенных выше, давайте поставим задачу линейной регрессии в следующем виде.

Пусть даны признаки $X$ и измерения $y$. 

Чтобы вывести все необходимые условные плотности, сделаем следующее предположение: существует идеальный набор параметров $\theta_{0}$, для которого выполняется
$$\forall_{i}: y_i = X_i\theta_0 + \epsilon_i$$
Здесь $\epsilon_i$ - шум, погрешность, распределенный по нормальному закону $N(0,\sigma^2)$. Для каждого $i$ и $j$ величины 
$\epsilon_i$ и $\epsilon_j$ считаются **независимыми**.

Вас не должно смущать именно такое предположение. Ведь выбирая предварительно линейную модель мы в тайне надеемся, что "правильный" ответ в виде $\theta_{0}$ существует. Шумы считаются нормально распределенными также вполне обоснованно, такая картина часто имеет место быть в различного рода задачах, где присутсвуют ошибки измерения.

Наша задача, как и прежде, каким-то образом оценить параметры $\theta_{0}$.

Применим технику максимального правдоподобия.

Выражение $y_i = X_i\theta_0 + \epsilon_i$ имеет нормальное распределение $N(X_i\theta_0,\sigma^2)$. Но так как заранее $\theta_0$ неизвестно, то мы имеем право предположить, что $y_i$ имеет нормальное распределение $N(X_i\theta,\sigma^2)$ с неизвестным $\theta$. Тем самым мы пришли к знакомой уже плотности $p(y_i|X_i,\theta) = N(X_i\theta,\sigma^2)$. 

Перейдем к совместной плотности $p(y|X,\theta) = \prod_{i=1}^{n} p(y_i|X_i,\theta)$. По свойству нормального распределения и с учетом езависимости измерений совместное распределение будет являться многомерным нормальным распределением со следующей плотностью:
$$ p(y|X,\theta) = \dfrac{1}{Z}e^{\dfrac{-\sum_i (y_i - X_i\theta)^{2}}{\sigma^2}} $$
здесь $Z$ не зависит от $X,y,\theta$

Рассматривая совместную плотность как функцию только от параметров $\theta$, мы получаем функцию правдоподобия:
$$L(\theta) = \dfrac{1}{Z}e^{\dfrac{-\sum_i (y_i - X_i\theta)^{2}}{\sigma^2}}$$
Теперь нам ничего не мешает перейти к поиску $\theta_{MLE}$:
$$\theta_{MLE}=\arg\max \ln L(\theta) = \arg\max -\dfrac{\sum_i (y_i - X_i\theta)^{2}}{\sigma^2}$$
Мы избавились от $Z$, так как $Z$ не зависит от параметров $\theta$.

Максимизация функции $f(x)$ эквивалента минимизации $-f(x)$. Тогда оптимальные параметры можно искать как:
$$\theta_{MLE}=\arg\min \dfrac{\sum_i (y_i - X_i\theta)^{2}}{\sigma^2}$$

Мы получили уже знакомую задачу нахождения минимума суммы квадратов.

Таким образом начальное предположение нормального распределения условной плотности $p(y_i|X_i,\theta)$ приводит нас к методу наименьших квадратов.


## Вывод бинарной кросс-энтропии через MLE

Регрессия приводит нас к МНК в случае нормального распределения. А что же классификация?

Пусть даны признаки $X$ и метки класса $y$. Здесь мы имеем дело уже не с плотностями, а с вероятностями ввиду дискретности $y$.

Рассмотрим логистическую регрессию. Как известно для *i*-того примера $\sigma(X_i\theta) = \dfrac{1}{1+e^{X_i\theta}}$ - это оценка вероятности принадлежности примера к классу $1$. Тогда условную вероятность $p(y_i|X_i,\theta)$ можно оценить как выражение:
$$p(y_i|X_i,\theta) = \sigma(X_i\theta)^y_{i}(1-\sigma(X_i\theta))^{1-y_{i}}$$

Действительно, если подставить в $p(y_i|X_i,\theta)$ значение $y_i=1$, то получим $p(1|X_i,\theta) = \sigma(X_i\theta)$, то есть вероятность принадлежности *i*-того примера к классу $1$; если подставить в $p(y_i|X_i,\theta)$ $y_i=0$, то получим $p(0|X_i,\theta) = 1-\sigma(X_i\theta)$, то есть вероятность принадлежности i-того примера к классу $0$.

Перейдем к совместной вероятности $p(y|X,\theta) = \prod_{i=1}^{n} p(y_i|X_i,\theta)$:
$$p(y|X,\theta) = \prod_{i=1}^{n} \sigma(X_i\theta)^y_{i}(1-\sigma(X_i\theta))^{1-y_{i}}$$

Рассматривая совместную вероятность как функцию только от параметров $\theta$, мы снова получаем функцию правдоподобия:
$$L(\theta) = \prod_{i=1}^{n} \sigma(X_i\theta)^y_{i}(1-\sigma(X_i\theta))^{1-y_{i}}$$
И снова нам ничего не мешает перейти к поиску $\theta_{MLE}$:
$$\theta_{MLE}=\arg\max \ln L(\theta) = \arg\max \sum_i y_{i}\ln \sigma(X_i\theta) + (1-y_{i})\ln(1-\sigma(X_i\theta))$$

Переходим к минимизации:
$$\theta_{MLE}=\arg\min \sum_i -y_{i}\ln \sigma(X_i\theta) - (1-y_{i})\ln(1-\sigma(X_i\theta))$$


И вновь мы получили знакомую функцию потерь, на этот раз бинарную кросс-энтропию.
